<a href="https://colab.research.google.com/github/khadijabendib/Agent-Intelligent/blob/main/Robotique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [86]:
import numpy as np

# Définir la taille de l'environnement
n_rows = 20
n_cols = 20

matrice = np.zeros((n_rows, n_cols), dtype=int)

# Ajouter les murs
matrice[0, :] = -1
matrice[-1, :] = -1
matrice[:, 0] = -1
matrice[:,9] = -1
matrice[9,:] = -1
matrice[9,7] = 0
matrice[9,16] = 0
matrice[5,9] = 0
matrice[16,9] = 0
matrice[:, -1] = -1

# Définir la position attendue
goal_pos = (15, 17)
matrice[goal_pos] = 10

#Générer une position aléatoire pour le robot
while True:
    robot_pos = np.random.randint(n_rows), np.random.randint(n_cols)
    if matrice[robot_pos] == -1:
        break
print(matrice)

[[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0 10

In [88]:
q_values = np.zeros((n_rows, n_cols, 4))

In [89]:
actions=['up','right','down','left']

In [90]:
#define a function that determines if the specified location is a terminal state
def is_terminal_state(current_row_index, current_column_index):
  #if the reward for this location is -1, then it is not a terminal state (i.e., it is a 'white square')
  if matrice[current_row_index, current_column_index] == 0 :
    return False
  else:
    return True

In [91]:
#define a function that will choose a random, non-terminal starting location
def get_starting_location():
  #get a random row and column index
  current_row_index = np.random.randint(n_rows)
  current_column_index = np.random.randint(n_cols)
  #continue choosing random row and column indexes until a non-terminal state is identified
  #(i.e., until the chosen state is a 'white square').
  while is_terminal_state(current_row_index, current_column_index):
    current_row_index = np.random.randint(n_rows)
    current_column_index = np.random.randint(n_cols)
  return current_row_index, current_column_index

In [76]:
def get_next_action(current_row_index, current_column_index, epsilon):
  #if a randomly chosen value between 0 and 1 is less than epsilon, 
  #then choose the most promising value from the Q-table for this state.
  if np.random.random() < epsilon:
    # print("hhhhhh")
    return np.argmax(q_values[current_row_index, current_column_index])
  else: #choose a random action
    return np.random.randint(4)

In [78]:
#define a function that will get the next location based on the chosen action
def get_next_location(current_row_index, current_column_index, action_index):
  new_row_index = current_row_index
  new_column_index = current_column_index
  if actions[action_index] == 'up' and current_row_index > 0:
    new_row_index -= 1
  elif actions[action_index] == 'right' and current_column_index < n_cols - 1:
    new_column_index += 1
  elif actions[action_index] == 'down' and current_row_index < n_rows - 1:
    new_row_index += 1
  elif actions[action_index] == 'left' and current_column_index > 0:
    new_column_index -= 1
  return new_row_index, new_column_index

In [79]:
#Define a function that will get the shortest path between any location within the warehouse that 
#the robot is allowed to travel and the item packaging location.
def get_shortest_path(start_row_index, start_column_index):
  #return immediately if this is an invalid starting location
  if is_terminal_state(start_row_index, start_column_index):
    return []
  else: #if this is a 'legal' starting location
    current_row_index, current_column_index = start_row_index, start_column_index
    shortest_path = []
    shortest_path.append([current_row_index, current_column_index])
    #continue moving along the path until we reach the goal (i.e., the item packaging location)
    while not is_terminal_state(current_row_index, current_column_index):
      #get the best action to take
      action_index = get_next_action(current_row_index, current_column_index, 1.)
      #move to the next location on the path, and add the new location to the list
      current_row_index, current_column_index = get_next_location(current_row_index, current_column_index, action_index)
      shortest_path.append([current_row_index, current_column_index])
      print("["+ str(current_row_index)  +", "+ str(current_column_index) +"]")
    return shortest_path

In [ ]:
#define training parameters
epsilon = 0.9 #the percentage of time when we should take the best action (instead of a random action)
discount_factor = 0.9 #discount factor for future rewards
learning_rate = 0.9 #the rate at which the AI agent should learn

#run through 1000 training episodes
for episode in range(100000):
  #get the starting location for this episode
  row_index, column_index = get_starting_location()
  print(episode)
  #continue taking actions (i.e., moving) until we reach a terminal state
  #(i.e., until we reach the item packaging area or crash into an item storage location)
  while not is_terminal_state(row_index, column_index):
    #choose which action to take (i.e., where to move next)
    action_index = get_next_action(row_index, column_index, epsilon)
    # print(action_index)
    #perform the chosen action, and transition to the next state (i.e., move to the next location)
    old_row_index, old_column_index = row_index, column_index #store the old row and column indexes
    row_index, column_index = get_next_location(row_index, column_index, action_index)
    
    #receive the reward for moving to the new state, and calculate the temporal difference
    reward = matrice[row_index, column_index]
    # print(reward)
    old_q_value = q_values[old_row_index, old_column_index, action_index]
    temporal_difference = reward + (discount_factor * np.max(q_values[row_index, column_index])) - old_q_value

    #update the Q-value for the previous state and action pair
    new_q_value = old_q_value + (learning_rate * temporal_difference)
    q_values[old_row_index, old_column_index, action_index] = new_q_value

print('Training complete!')

In [82]:
print(matrice)

[[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0 10

In [83]:
#display a few shortest paths
print(get_shortest_path(16, 16)) #starting at row 3, column 9
# print(get_shortest_path(5, 0)) #starting at row 5, column 0
print(get_shortest_path(17, 17)) #starting at row 9, column 5

[15, 16]
[15, 17]
[[16, 16], [15, 16], [15, 17]]
[16, 17]
[15, 17]
[[17, 17], [16, 17], [15, 17]]
